In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

role = sagemaker.get_execution_role()

In [7]:
from sagemaker.tensorflow import TensorFlow

instance_type = 'ml.g4dn.2xlarge'
instance_count = 4

model_dir = "/opt/ml/model"

distributions={
    "smdistributed": {
        "modelparallel": {
            "enabled":True,
            "parameters": {
                "partitions": 4,
                "microbatches": 4,
                "placement_strategy": "spread",
                "pipeline": "interleaved",
                "optimize": "speed",
            }
        },
    },
    "mpi": {
        "enabled": True,
        "processes_per_host": 8,
    }
}

hyperparameters = {'epochs': 100, 'batch-size': 8}

estimator = TensorFlow(
                       source_dir='./code',
                       entry_point='train.py',
                       base_job_name='3dcnn-tf',
                       role=role,
                       framework_version='2.2',
                       py_version='py37',
                       hyperparameters=hyperparameters,
                       instance_count=instance_count,
                       instance_type=instance_type,
                       model_dir=model_dir,
                       distribution=distributions,
                       session=sagemaker_session)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [8]:
estimator.fit({"training": "s3://fastvision.ai/segmented_data/LUNA16_segmented_4mm"})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: 3dcnn-tf-2023-08-06-18-12-00-894


Using provided s3_resource
2023-08-06 18:12:01 Starting - Starting the training job...
2023-08-06 18:12:16 Starting - Preparing the instances for training......
2023-08-06 18:13:31 Downloading - Downloading input data......
2023-08-06 18:14:11 Training - Downloading the training image......
2023-08-06 18:15:12 Training - Training image download completed. Training in progress..2023-08-06 18:15:35.646605: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-08-06 18:15:35.646771: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-08-06 18:15:35.672105: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2023-08-06 18:15:36,885 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2023-08-06 18:15:37,091 sage

UnexpectedStatusException: Error for Training job 3dcnn-tf-2023-08-06-18-12-00-894: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "mpirun --host algo-1:8,algo-2:8,algo-3:8,algo-4:8 -np 32 --allow-run-as-root --display-map --tag-output -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -bind-to none -map-by slot -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca btl_vader_single_copy_mechanism none -x NCCL_MIN_NRINGS=4 -x NCCL_SOCKET_IFNAME=eth0 -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -x LD_PRELOAD=/usr/local/lib/python3.7/site-packages/gethostname.cpython-37m-x86_64-linux-gnu.so -x SM_HOSTS -x SM_NETWORK_INTERFACE_NAME -x SM_HPS -x SM_USER_ENTRY_POINT -x SM_FRAMEWORK_PARAMS -x SM_RESOURCE_CONFIG -x SM_INPUT_DATA_CONFIG -x SM_OUTPUT_DATA_DIR -x SM_CHANNELS -x SM_CURRENT_HOST -x SM_MODULE_NAME -x SM_LOG_LEVEL -x SM_FRAMEWORK_MODULE -x SM_INPUT_DIR -x SM_INPUT_CONFIG_DIR -x SM_OUTPUT_DIR -x SM_NUM_CPUS -x SM_